Models are trained on different categories

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
from tqdm import tqdm
import json
import os
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, ConfusionMatrixDisplay
from hpsklearn import svc
from sklearn.model_selection import train_test_split
from hyperopt import hp

# Prep data

In [2]:
def embed_dataset(dataset_text):
    nlp = spacy.load("spacy-twitter")
    encoded = np.array([nlp(text).vector for text in dataset_text])
    return encoded.tolist()

In [3]:
def get_dataset(name):
    dataset = pd.read_csv(f'datasets\\{name}.csv')
    dataset.rename(columns = {"Unnamed: 0":"entry"}, inplace=True)
    dataset['e_text'] = embed_dataset(dataset['text'])
    return dataset

pheme = get_dataset("pheme")
twitter = get_dataset("twitter")

# Train models  

SVMs trained on each category

In [4]:
from hpsklearn import HyperoptEstimator
from hpsklearn import svc

def optimize_model(model, X_train, y_train):
    mod = HyperoptEstimator(classifier=model,
                            preprocessing=[],
                            max_evals=1,
                            trial_timeout=20,
                            verbose=False)
    mod.fit(X_train, y_train, random_state=42)
    #print(mod.best_model())
    return mod

def evaluate_model(model, X_test, y_test):
    pred_y = model.predict(X_test)
    acc_mod = accuracy_score(y_test, pred_y)
    print("Accuracy:", float("{0:.2f}".format(acc_mod*100)), "%")
    f1_mod = f1_score(y_test, pred_y, average="macro")
    print("F1:", float("{0:.2f}".format(f1_mod*100)), "%")
    cm = confusion_matrix(y_test, pred_y)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["true", "false"])
    disp.plot()
    plt.show() 

In [5]:
#Multiple datasets
confidence_threshold = 0.2
size_threshold = 50
train_set = pheme

def train_svms(dataset, confidence_threshold, size_threshold, train_set):
    file_name = f"{dataset}_cats\\{dataset}_categories_organised.json"
    f = open(file_name)
    data = json.load(f)
    trained_svms = {}
    for key in data.keys():
        svm_name = f"svm_{key}"
        cat_entries = [int(i) for i in data[key].keys() if data[key][i] > confidence_threshold]
        if len(cat_entries) < size_threshold:
            print(f"Skipped category: {key} due to low numbers")
            continue
        all_in_cat = train_set.filter(axis=0, items=cat_entries)
        X_train = all_in_cat.drop('target', axis=1)
        y_train = all_in_cat['target']
        #print(np.unique(all_in_cat["target"]))
        if (len(np.unique(all_in_cat["target"])) <= 1):
            print(f"Skipped category: {key} due to class issues")
            continue
        X_train_text = np.array([text for text in X_train['e_text']])
        try:
            svm = optimize_model(svc(name=svm_name, random_state=42, probability=True), X_train_text, y_train)
        except:
            print(f"error training {key} svm, skipping")
            continue
        trained_svms[key] = svm
        print(f"Created SVM trained in category: {key}")
    return trained_svms

#trained_svms = train_svms("pheme", confidence_threshold, size_threshold)

In [6]:
#model_list = [svc(name="name", random_state=42, probability=True), knn(), etc]

def train_models(dataset, confidence_threshold, size_threshold, train_set, model_list):
    file_name = f"{dataset}_cats\\{dataset}_categories_organised.json"
    f = open(file_name)
    data = json.load(f)
    trained_models = {}
    for key in data.keys():
        cat_entries = [int(i) for i in data[key].keys() if data[key][i] > confidence_threshold]
        if len(cat_entries) < size_threshold:
            print(f"Skipped category: {key} due to low numbers")
            continue
        all_in_cat = train_set.filter(axis=0, items=cat_entries)
        X_train = all_in_cat.drop('target', axis=1)
        y_train = all_in_cat['target']
        if (len(np.unique(all_in_cat["target"])) <= 1):
            print(f"Skipped category: {key} due to class issues")
            continue
        X_train_text = np.array([text for text in X_train['e_text']])
        trained_models[key] = {}
        for model_name, model in model_list:
            try:
                m = optimize_model(model, X_train_text, y_train)
                mod = m.best_model()['learner'].fit(X_train_text, y_train)
            except:
                print(f"Error training {model_name} in category {key}, skipping")
                continue
            trained_models[key][model_name] = mod
    return trained_models

In [7]:
file_name = "pheme_cats\\pheme_categories_organised.json"
f = open(file_name)
data = json.load(f)

confidence_threshold = 0.5
for key in data.keys():
    entries = [int(i) for i in data[key].keys() if data[key][i] > confidence_threshold]
    all_in_cat = pheme.filter(axis=0, items=entries)
    t = 0
    f=0
    for e in all_in_cat["target"]:
        if e:
            t += 1
        else:
            f += 1
    print(key, len(entries), len(all_in_cat))
    #print(t, f)

People & Society 165 165
Arts & Entertainment 154 154
Law & Government 86 86
News 262 262
Food & Drink 62 62
Sensitive Subjects 579 579
Online Communities 26 26
Internet & Telecom 11 11
Computers & Electronics 10 10
Health 45 45
Pets & Animals 32 32
Reference 11 11
Adult 3 3
Business & Industrial 23 23
Books & Literature 4 4
Jobs & Education 5 5
Shopping 21 21
Beauty & Fitness 6 6
Autos & Vehicles 7 7
Science 18 18
Finance 9 9
Travel & Transportation 16 16
Hobbies & Leisure 8 8
Sports 64 64
Games 4 4
Home & Garden 0 0
Real Estate 0 0


In [8]:
# ONLY SVMS
def predict_points(trained_svms, test_cats, test_data):
    test_points = test_data
    test_file_name = test_cats
    f = open(test_file_name)
    test_data = json.load(f)
    f.close()
    final_predictions = []
    final_predictions2 = []
    for i in tqdm(range(len(test_points))):
        point = test_points.iloc[i]
        point_text = np.array([text for text in point['e_text']])
        topics = test_data[str(point["entry"])]
        topic_weights = {}
        for topic in topics:
            main_topic = topic.split("/")[1]
            if topics[topic] < confidence_threshold or main_topic not in trained_svms.keys():
                continue
            if main_topic in topic_weights:
                topic_weights[main_topic] += topics[topic]
            else:
                topic_weights[main_topic] = topics[topic]
        #print(topic_weights) 
        model_predictions = []
        for topic in topic_weights:
            model = trained_svms[topic]
            pred = model.predict(point_text.reshape(1,-1))
            model_predictions.append((pred[0], topic_weights[topic]))
            #model_predictions.append((pred[0], 1))
        #print(model_predictions)
        true_mark = 0
        false_mark = 0
        for pred, score in model_predictions:    
            if pred == True:
                true_mark += score
            else:
                false_mark += score
        #print(mark)
        if (true_mark > false_mark):
            final_predictions.append(True)
        else:
            final_predictions.append(False)
        max = 0
        final_pred = True
        for pred, score in model_predictions:
            if score > max:
                final_pred = pred
        final_predictions2.append(final_pred)
    return final_predictions, final_predictions2
             
def check_score(test, pred):
    acc = accuracy_score(test, pred)
    f1 = f1_score(test, pred, average="macro")
    return float("{0:.2f}".format(acc*100)), float("{0:.2f}".format(f1*100))

In [32]:
# MULTIPLE MODELS
from sklearn.ensemble import VotingClassifier

def hard_vote(model_predictions):
    category_votes = []
    for category in model_predictions.keys():
        predictions = model_predictions[category]
        vote = 0
        for pred, model, weight in predictions:
            if pred[0][0] < pred[0][1]:
                vote += 1
            else: vote -= 1 
        if vote > 0:
            category_votes.append((category, True, weight))
        else:
            category_votes.append((category, False, weight))
    score = 0
    for category, vote, weight in category_votes:
        if vote == True: score += weight
        else: score -= weight
    if score > 0: return True
    else: return False

def soft_vote(model_predictions):
    category_votes = []
    for category in model_predictions.keys():
        predictions = model_predictions[category]
        max_vote = [True, 0, "", 0]
        for pred, model, weight in predictions:
            false_weight = pred[0][0]
            true_weight = pred[0][1]
            if false_weight < true_weight:
                vote = [True, true_weight, model, weight]
            else:
                vote = [False, false_weight, model, weight]
            category_votes.append(vote)
    score = 0
    for pred, confidence, model, weight in category_votes:
        vote_weight = (0.5*confidence + 1*weight)
        if pred == True: score += vote_weight
        else: score -= vote_weight
    if score > 0: return True
    else: return False

def predict_points_mutiple_models(trained_models, test_cats, test_data):
    test_points = test_data
    test_file_name = test_cats
    f = open(test_file_name)
    test_data = json.load(f)
    f.close()
    final_predictions_hard = []
    final_predictions_soft = []
    for i in tqdm(range(len(test_points))):
        point = test_points.iloc[i]
        point_text = np.array([text for text in point['e_text']])
        topics = test_data[str(point["entry"])]
        topic_weights = {}
        for topic in topics:
            main_topic = topic.split("/")[1]
            if topics[topic] < confidence_threshold or main_topic not in trained_models.keys():
                continue
            if main_topic in topic_weights:
                topic_weights[main_topic] += topics[topic]
            else:
                topic_weights[main_topic] = topics[topic]
        model_predictions = {}
        for topic in topic_weights:
            models = trained_models[topic]
            model_predictions[topic] = []
            for model in models:
                mod = models[model]
                pred = mod.predict_proba(point_text.reshape(1,-1))
                model_predictions[topic].append((pred, model, topic_weights[topic]))
        #vote
        prediction_hard = hard_vote(model_predictions)
        prediction_soft = soft_vote(model_predictions)
        final_predictions_hard.append(prediction_hard)
        final_predictions_soft.append(prediction_soft)
    return final_predictions_hard, final_predictions_soft

def check_score(test, pred):
    acc = accuracy_score(test, pred)
    f1 = f1_score(test, pred, average="macro")
    return float("{0:.2f}".format(acc*100)), float("{0:.2f}".format(f1*100))   

#a,b = predict_points_mutiple_models(models, "pheme_categories.json", X_val)     

# Pipeline

In [21]:
# ONLY SVMS
def get_results(train_dataset, confidence_threshold, size_threshold, train, tests): 
    X_train, X_val, y_train, y_val = train_test_split(train.drop("target", axis=1), train["target"], train_size=0.8, stratify=train["target"]) 
    train_set = pd.concat([X_train, y_train], axis=1)
    svms = train_svms(train_dataset, confidence_threshold, size_threshold, train_set)

    results = []
    train_cats = f"{train_dataset}_categories.json"
    predictions1, predictions2 = predict_points(svms, train_cats, X_val)
    results.append(check_score(predictions1, y_val))
    
    for test_set, test_cat, test_name in tests:
        test_cat_file = f"{test_cat}_categories.json"
        test_data = test_set.drop("target", axis=1)
        test_target = test_set["target"]
        predictions1, predictions2 = predict_points(svms, test_cat_file, test_data)
        results.append((test_name, check_score(predictions1, test_target)))
    return results

def run_tests(tests, confidence_threshold, size_threshold):
    test_results = []
    for i in tqdm(range(len(tests))):
        t = tests.copy()
        train = t.pop(i)
        test_results.append((train[2], get_results(train[1], confidence_threshold, size_threshold, train[0], t)))
    return test_results

In [22]:
# MULTIPLE MODELS
def get_results_multi(train_dataset, confidence_threshold, size_threshold, train, tests, model_list): 
    X_train, X_val, y_train, y_val = train_test_split(train.drop("target", axis=1), train["target"], train_size=0.8, stratify=train["target"]) 
    train_set = pd.concat([X_train, y_train], axis=1)
    models = train_models(train_dataset, confidence_threshold, size_threshold, train_set, model_list)

    results = []
    train_cats = f"{train_dataset}_categories.json"
    predictions_hard, predictions_soft = predict_points_mutiple_models(models, train_cats, X_val)
    results.append(check_score(predictions_hard, y_val), check_score(predictions_soft, y_val))
    
    for test_set, test_cat, test_name in tests:
        test_cat_file = f"{test_cat}_categories.json"
        test_data = test_set.drop("target", axis=1)
        test_target = test_set["target"]
        predictions_hard, predictions_soft = predict_points_mutiple_models(models, test_cat_file, test_data)
        results.append((test_name, check_score(predictions_hard, test_target), check_score(predictions_soft, test_target)))
    return results

def run_tests_multi(tests, confidence_threshold, size_threshold, model_list):
    test_results = []
    for i in tqdm(range(len(tests))):
        t = tests.copy()
        train = t.pop(i)
        test_results.append((train[2], get_results_multi(train[1], confidence_threshold, size_threshold, train[0], t, model_list)))
    return test_results

In [23]:
from hpsklearn import svc, logistic_regression, k_neighbors_classifier, ada_boost_classifier, decision_tree_classifier, mlp_classifier
model_list = [
    ("SVC", svc("SVC", random_state=42, probability=True)),
    ("KNN", k_neighbors_classifier("knn")),
    ("Logistic Regression", logistic_regression("LR", random_state=42, solver="saga", penalty=hp.choice("penalty", {None, "l1", "l2"}))),
    ("Adaboost", ada_boost_classifier("adaDT", base_estimator=decision_tree_classifier("DT"), random_state=42)),
    ("MLP", mlp_classifier("MLP", random_state=42))
]

In [13]:
pheme = get_dataset("pheme")
twitter = get_dataset("twitter")
twitter15 = twitter.iloc[:1491]
twitter16 = twitter.iloc[1491:]
tests = [[pheme, "pheme", "PHEME"], [twitter15, "twitter", "twitter15"], [twitter16, "twitter", "twitter16"]]


In [15]:
tests = [[pheme, "pheme", "PHEME"], [twitter15, "twitter", "twitter15"], [twitter16, "twitter", "twitter16"]]
results1 = run_tests(tests, 0.2, 50)
results2 = run_tests(tests, 0.5, 20)
results3 = run_tests(tests, 0.2, 200)
results4 = run_tests(tests, 0, 100)

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:02<00:00,  2.46s/trial, best loss: 0.3316195372750642]
{'learner': SVC(C=1.27806368437018, coef0=0.48726712591817056, degree=1, gamma='auto',
    kernel='linear', probability=True, random_state=42, shrinking=False,
    tol=0.0006615201746896151), 'preprocs': (), 'ex_preprocs': ()}
Created SVM trained in category: Sensitive Subjects/War & Conflict
100%|██████████| 25/25 [00:05<00:00,  5.37s/trial, best loss: 0.4246913580246914]
{'learner': SVC(C=0.9657655830484548, coef0=0.6786722363807116,
    decision_function_shape='ovo', degree=2, probability=True, random_state=42,
    shrinking=False, tol=1.4737120975701307e-05), 'preprocs': (), 'ex_preprocs': ()}
Created SVM trained in category: News
100%|██████████| 25/25 [00:01<00:00,  1.97s/trial, best loss: 0.3113207547169812]
{'learner': SVC(C=1.174510237626359, coef0=0.02669401762935497,
    decision_function_shape='ovo', degree=1, gamma='auto', probability=True,
    random_state=42, shrinking=False, tol=0.00929969

 33%|███▎      | 1/3 [12:29<24:58, 749.48s/it]

100%|██████████| 25/25 [00:01<00:00,  1.72s/trial, best loss: 0.125]
{'learner': SVC(C=1.1945646294220587, coef0=0.7884022933921573, degree=1, kernel='linear',
    probability=True, random_state=42, tol=0.0005025238209911815), 'preprocs': (), 'ex_preprocs': ()}
Created SVM trained in category: People & Society
100%|██████████| 25/25 [00:01<00:00,  1.72s/trial, best loss: 0.24]
{'learner': SVC(C=0.6492912940993498, coef0=0.5441071721230211,
    decision_function_shape='ovo', degree=2, probability=True, random_state=42,
    shrinking=False, tol=6.834361122169257e-05), 'preprocs': (), 'ex_preprocs': ()}
Created SVM trained in category: Arts & Entertainment
100%|██████████| 25/25 [00:01<00:00,  1.73s/trial, best loss: 0.23809523809523814]
{'learner': SVC(C=0.8382306246268949, coef0=0.1803690144099015, degree=4, probability=True,
    random_state=42, tol=0.0006357388371780233), 'preprocs': (), 'ex_preprocs': ()}
Created SVM trained in category: Law & Government
100%|██████████| 25/25 [00:01

 67%|██████▋   | 2/3 [24:06<11:58, 718.81s/it]

...

 33%|███▎      | 1/3 [13:41<27:23, 821.74s/it]

100%|██████████| 25/25 [00:01<00:00,  1.79s/trial, best loss: 0.1785714285714286]
{'learner': SVC(C=0.9689573367316257, coef0=0.4308797345901485,
    decision_function_shape='ovo', gamma='auto', kernel='linear',
    probability=True, random_state=42, tol=2.3634845999761477e-05), 'preprocs': (), 'ex_preprocs': ()}
Created SVM trained in category: People & Society
100%|██████████| 25/25 [00:01<00:00,  1.78s/trial, best loss: 0.23529411764705888]
{'learner': SVC(C=0.708658334469585, coef0=0.4802867330432232,
    decision_function_shape='ovo', degree=4, probability=True, random_state=42,
    shrinking=False, tol=2.3984789710312818e-05), 'preprocs': (), 'ex_preprocs': ()}
Created SVM trained in category: Arts & Entertainment
100%|██████████| 25/25 [00:01<00:00,  1.75s/trial, best loss: 0.19999999999999996]
{'learner': SVC(C=1.1842439006344645, coef0=0.9222532817907875,
    decision_function_shape='ovo', degree=4, gamma='auto', kernel='linear',
    probability=True, random_state=42, tol=0.00

 67%|██████▋   | 2/3 [23:59<11:41, 701.71s/it]

100%|██████████| 25/25 [00:01<00:00,  1.63s/trial, best loss: 0.18918918918918914]
{'learner': SVC(C=1.1854487404071743, coef0=0.4616991803791799, degree=5, kernel='poly',
    probability=True, random_state=42, tol=0.006744868923094926), 'preprocs': (), 'ex_preprocs': ()}
Created SVM trained in category: People & Society
100%|██████████| 25/25 [00:01<00:00,  1.64s/trial, best loss: 0.18604651162790697]
{'learner': SVC(C=0.8238837122820875, coef0=0.29430990881146823, degree=5, kernel='poly',
    probability=True, random_state=42, shrinking=False,
    tol=0.00015236716068676753), 'preprocs': (), 'ex_preprocs': ()}
Created SVM trained in category: Arts & Entertainment
100%|██████████| 25/25 [00:01<00:00,  1.65s/trial, best loss: 0.09090909090909094]
{'learner': SVC(C=1.0288633092207546, coef0=0.6241710851772245,
    decision_function_shape='ovo', degree=4, kernel='poly', probability=True,
    random_state=42, tol=1.7548258743746086e-05), 'preprocs': (), 'ex_preprocs': ()}
Created SVM trai

100%|██████████| 3/3 [33:56<00:00, 678.73s/it]


In [14]:
X_train, X_val, y_train, y_val = train_test_split(pheme.drop("target", axis=1), pheme["target"], train_size=0.8, stratify=pheme["target"]) 
train_set = pd.concat([X_train, y_train], axis=1)
models = train_models("pheme", 0.2, 50, train_set, model_list)
a = predict_points_mutiple_models(models, "pheme_categories.json", X_val)

  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


100%|██████████| 1/1 [00:02<00:00,  2.13s/trial, best loss: 0.6338028169014085]


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


...

c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_logistic.py:1192: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


100%|██████████| 1/1 [00:01<00:00,  1.93s/trial, best loss: 0.23076923076923073]
Skipped category: Games due to low numbers
Skipped category: Home & Garden due to low numbers
Skipped category: Real Estate due to low numbers


100%|██████████| 1285/1285 [00:09<00:00, 131.00it/s]


In [33]:
a,b = predict_points_mutiple_models(models, "pheme_categories.json", X_val)

100%|██████████| 1285/1285 [00:15<00:00, 84.32it/s] 


In [34]:
check_score(b, y_val)

(69.26, 62.12)

In [39]:
multi_result1 = run_tests_multi(tests, 0.2, 50, model_list)
multi_result2 = run_tests_multi(tests, 0.5, 20, model_list)
multi_result3 = run_tests_multi(tests, 0.2, 200, model_list)
multi_result4 = run_tests_multi(tests, 0, 100, model_list)

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:02<00:00,  2.30s/trial, best loss: 0.3282442748091603]
{'learner': SVC(C=1.0234425670465672, coef0=0.28546581040736607, degree=2, probability=True,
    random_state=42, shrinking=False, tol=1.2113347263682842e-05), 'preprocs': (), 'ex_preprocs': ()}
100%|██████████| 25/25 [00:01<00:00,  1.78s/trial, best loss: 0.3078880407124682]
{'learner': KNeighborsClassifier(leaf_size=34, metric='cityblock', n_jobs=1, n_neighbors=11,
                     p=4.012983869021016), 'preprocs': (), 'ex_preprocs': ()}


job exception: The 'penalty' parameter of LogisticRegression must be a str among {'l1', 'l2', 'elasticnet', 'none' (deprecated)} or None. Got 'None' instead.

  0%|          | 0/3 [01:52<?, ?it/s]

  0%|          | 0/1 [00:01<?, ?trial/s, best loss=?]
Error training Logistic Regression in category Sensitive Subjects/War & Conflict, skipping
100%|██████████| 25/25 [00:04<00:00,  4.40s/trial, best loss: 0.3282442748091603]


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


...

job exception: The 'penalty' parameter of LogisticRegression must be a str among {'l1', 'elasticnet', 'none' (deprecated), 'l2'} or None. Got 'None' instead.

 67%|██████▋   | 2/3 [3:32:34<1:20:27, 4827.77s/it]

  0%|          | 0/1 [00:01<?, ?trial/s, best loss=?]
Error training Logistic Regression in category Sports, skipping
100%|██████████| 25/25 [00:01<00:00,  1.72s/trial, best loss: 0.0]
{'learner': AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=DecisionTreeClassifier(max_features=0.9152610638484507,
                                                         random_state=2,
                                                         splitter='random'),
                   learning_rate=0.0005935578813064877, n_estimators=123,
                   random_state=42), 'preprocs': (), 'ex_preprocs': ()}


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


100%|██████████| 25/25 [00:01<00:00,  1.87s/trial, best loss: 0.16666666666666663]
{'learner': MLPClassifier(activation='logistic', alpha=0.003257938755162686,
              beta_1=0.8856396471339226, beta_2=0.9688481055426172,
              epsilon=4.886158113583965e-06, learning_rate='invscaling',
              learning_rate_init=0.03259829716394281, max_fun=11789,
              max_iter=345, momentum=0.9190320513620664, n_iter_no_change=30,
              power_t=0.16457669672583342, random_state=42, solver='sgd',
              tol=0.00851028483929724, validation_fraction=0.1531696803766395), 'preprocs': (), 'ex_preprocs': ()}
Skipped category: Sensitive Subjects/Firearms & Weapons due to low numbers
Skipped category: Games due to low numbers
Skipped category: Home & Garden due to low numbers
Skipped category: Sensitive Subjects/Recreational Drugs due to low numbers
Skipped category: Real Estate due to low numbers


100%|██████████| 3/3 [3:37:31<00:00, 4350.64s/it]  


In [43]:
multi_result4

[('PHEME',
  [(79.84, 75.71),
   ('twitter15', (50.77, 44.04)),
   ('twitter16', (57.53, 50.81))]),
 ('twitter15',
  [(76.25, 75.78), ('PHEME', (60.98, 51.52)), ('twitter16', (55.2, 48.06))]),
 ('twitter16',
  [(75.61, 74.77), ('PHEME', (61.37, 59.35)), ('twitter15', (57.21, 54.15))])]

In [31]:
a = predict_points_mutiple_models(models, "pheme_categories.json", X_val)
check_score(a, y_val)

  0%|          | 0/1285 [00:00<?, ?it/s]

100%|██████████| 1285/1285 [00:36<00:00, 34.93it/s]


(80.54, 77.7)

In [15]:
tests = [[pheme, "pheme", "PHEME"], [twitter15, "twitter", "twitter15"], [twitter16, "twitter", "twitter16"]]
r = get_results_multi("pheme", 0.2, 50, pheme, tests, model_list)

100%|██████████| 1/1 [00:02<00:00,  2.51s/trial, best loss: 0.4974619289340102]
{'learner': SVC(C=0.874214966831998, coef0=0.6997840052684804,
    decision_function_shape='ovo', gamma='auto', kernel='poly',
    probability=True, random_state=42, shrinking=False,
    tol=1.4177669173970015e-05), 'preprocs': (), 'ex_preprocs': ()}
100%|██████████| 1/1 [00:02<00:00,  2.01s/trial, best loss: 0.31725888324873097]
{'learner': KNeighborsClassifier(algorithm='ball_tree', leaf_size=39, metric='manhattan',
                     n_jobs=1, n_neighbors=14, p=1.2851407547643645,
                     weights='distance'), 'preprocs': (), 'ex_preprocs': ()}
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

job exception: The 'penalty' parameter of LogisticRegression must be a str among {'elasticnet', 'none' (deprecated), 'l1', 'l2'} or None. Got 'None' instead.



  0%|          | 0/1 [00:01<?, ?trial/s, best loss=?]
Error training Logistic Regression in category Sensitive Subjects/War & Conflict, skipping
100%|██████████| 1/1 [00:03<00:00,  3.66s/trial, best loss: 0.4568527918781726]


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


{'learner': AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                         max_features='sqrt',
                                                         random_state=3,
                                                         splitter='random'),
                   learning_rate=0.15972525054853579, n_estimators=206,
                   random_state=42), 'preprocs': (), 'ex_preprocs': ()}


...

c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (280) reached and the optimization hasn't converged yet.
  warnings.warn(


{'learner': MLPClassifier(activation='identity', alpha=0.001782930675198739,
              beta_1=0.9718440833270774, beta_2=0.9850006618539915,
              epsilon=3.845129280197866e-07, learning_rate='adaptive',
              learning_rate_init=0.08459547138701909, max_fun=29546,
              max_iter=280, momentum=0.9011269643896735, n_iter_no_change=30,
              power_t=0.6550398515855504, random_state=42, solver='sgd',
              tol=0.009316698812528867, validation_fraction=0.1566911883066295), 'preprocs': (), 'ex_preprocs': ()}


c:\Users\willc\miniconda3\envs\test\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (280) reached and the optimization hasn't converged yet.
  warnings.warn(


Skipped category: Beauty & Fitness due to low numbers
Skipped category: Science due to low numbers
Skipped category: Computers & Electronics due to low numbers
Skipped category: Sensitive Subjects/Recreational Drugs due to low numbers
Skipped category: Shopping due to low numbers
Skipped category: Internet & Telecom due to low numbers
Skipped category: Adult due to low numbers
Skipped category: Finance due to low numbers
Skipped category: Sensitive Subjects/Self-Harm due to low numbers


100%|██████████| 817/817 [00:10<00:00, 79.26it/s] 


In [45]:
# Multi baseline
from sklearn.ensemble import VotingClassifier

def evaluate_baseline(model, X_test, y_test):
    pred_y = model.predict(X_test)
    acc_mod = accuracy_score(y_test, pred_y)
    f1_mod = f1_score(y_test, pred_y, average="macro")
    return float("{0:.2f}".format(acc_mod*100)), float("{0:.2f}".format(f1_mod*100))


def run_baseline_tests(tests, model_list):
    test_results = []
    for i in tqdm(range(len(tests))):
        t = tests.copy()
        train = t.pop(i)
        test_results.append((train[2], get_baseline_results(train[0], tests, model_list)))
    return test_results

def get_baseline_results(train, tests, model_list): 
    X_train, X_val, y_train, y_val = train_test_split(train.drop("target", axis=1), train["target"], train_size=0.8, stratify=train["target"]) 
    X_train_text = np.array([text for text in X_train['e_text']])
    X_val_text = np.array([text for text in X_val['e_text']])
    models = []
    for model_name, model in model_list:
        baseline_model = optimize_model(model, X_train_text, y_train)
        sk_model = baseline_model.best_model()["learner"].fit(X_train_text, y_train)
        models.append((model_name, sk_model))

    vc_hard = VotingClassifier(estimators=models, voting="hard")
    vc_hard = vc_hard.fit(X_train_text, y_train)
    vc_soft = VotingClassifier(estimators=models, voting="soft")
    vc_soft.fit(X_train_text, y_train)

    results = []
    results.append((evaluate_baseline(vc_hard, X_val_text, y_val), evaluate_baseline(vc_soft, X_val_text, y_val)))
    for test_set, test_cat, test_name in tests:
        test_data = test_set.drop("target", axis=1)
        test_data_text = np.array([text for text in test_data['e_text']])
        test_target = test_set["target"]
        results.append((test_name, evaluate_baseline(vc_hard, test_data_text, test_target), evaluate_baseline(vc_soft, test_data_text, test_target),))
    return results

In [12]:
def evaluate_baseline(model, X_test, y_test):
    pred_y = model.predict(X_test)
    acc_mod = accuracy_score(y_test, pred_y)
    f1_mod = f1_score(y_test, pred_y, average="macro")
    return float("{0:.2f}".format(acc_mod*100)), float("{0:.2f}".format(f1_mod*100))


def run_baseline_tests(tests):
    test_results = []
    for i in tqdm(range(len(tests))):
        t = tests.copy()
        train = t.pop(i)
        test_results.append((train[2], get_baseline_results(train[0], tests)))
    return test_results

def get_baseline_results(train, tests): 
    X_train, X_val, y_train, y_val = train_test_split(train.drop("target", axis=1), train["target"], train_size=0.8, stratify=train["target"]) 
    X_train_text = np.array([text for text in X_train['e_text']])
    X_val_text = np.array([text for text in X_val['e_text']])
    baseline = optimize_model("svm", X_train_text, y_train)
    results = []
    results.append(evaluate_baseline(baseline, X_val_text, y_val))
    for test_set, test_cat, test_name in tests:
        test_data = test_set.drop("target", axis=1)
        test_data_text = np.array([text for text in test_data['e_text']])
        test_target = test_set["target"]
        results.append((test_name, evaluate_baseline(baseline, test_data_text, test_target)))
    return results

baseline_results = run_baseline_tests(tests)

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:06<00:00,  6.66s/trial, best loss: 0.15369649805447472]
{'learner': SVC(C=1.2007340688965829, coef0=0.41195154588714744, degree=5, kernel='poly',
    probability=True, random_state=42, tol=0.0021174726338347707), 'preprocs': (), 'ex_preprocs': ()}


 33%|███▎      | 1/3 [03:05<06:11, 185.73s/it]

100%|██████████| 25/25 [00:01<00:00,  1.97s/trial, best loss: 0.20502092050209209]
{'learner': SVC(C=0.5853585639125682, coef0=0.9129795392049047,
    decision_function_shape='ovo', degree=5, kernel='poly', probability=True,
    random_state=42, tol=2.2560165272510383e-05), 'preprocs': (), 'ex_preprocs': ()}


 67%|██████▋   | 2/3 [03:59<01:48, 108.03s/it]

100%|██████████| 25/25 [00:01<00:00,  1.88s/trial, best loss: 0.1984732824427481]
{'learner': SVC(C=0.9906379338992914, coef0=0.5764761568576748, degree=2, kernel='poly',
    probability=True, random_state=42, shrinking=False,
    tol=8.617804258277811e-05), 'preprocs': (), 'ex_preprocs': ()}


100%|██████████| 3/3 [04:46<00:00, 95.48s/it] 


In [13]:
baseline_results

[('PHEME',
  [(83.5, 82.24),
   ('PHEME', (94.09, 93.64)),
   ('twitter15', (50.17, 47.63)),
   ('twitter16', (61.69, 59.78))]),
 ('twitter15',
  [(81.61, 81.6),
   ('PHEME', (55.7, 51.97)),
   ('twitter15', (95.91, 95.91)),
   ('twitter16', (62.3, 60.07))]),
 ('twitter16',
  [(74.39, 74.39),
   ('PHEME', (61.29, 60.87)),
   ('twitter15', (56.2, 55.43)),
   ('twitter16', (87.52, 87.52))])]